In [1]:
import numpy as np
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from pandas import Series

Using TensorFlow backend.


In [2]:
# load dataset and separate dates
btc_df = read_csv('../data/processed/price_trend_data.csv')
dates = btc_df.date
dataset = btc_df[['log_returns', 'log_trend_chg']]

# normalize the dataset
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [3]:
# reshape the data so that y is the next time step
def series_to_supervised(data, n_in=10, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
train = series_to_supervised(train)

In [68]:
train

,var1(t-10),var2(t-10),var3(t-10),var4(t-10),var5(t-10),var6(t-10),var7(t-10),var8(t-10),var9(t-10),var10(t-10),...,var2653(t),var2654(t),var2655(t),var2656(t),var2657(t),var2658(t),var2659(t),var2660(t),var2661(t),var2662(t)
40,0.598583,0.483973,0.556999,0.477577,0.618537,0.354642,0.692000,0.462193,0.558725,0.549306,...,0.313235,0.591679,0.393443,0.297453,0.542064,0.406752,0.507529,0.625275,0.485153,0.412742
41,0.556999,0.477577,0.618537,0.354642,0.692000,0.462193,0.558725,0.549306,0.139964,0.367555,...,0.393443,0.297453,0.542064,0.406752,0.507529,0.625275,0.485153,0.412742,0.446680,0.356187
42,0.618537,0.354642,0.692000,0.462193,0.558725,0.549306,0.139964,0.367555,0.620495,0.425468,...,0.542064,0.406752,0.507529,0.625275,0.485153,0.412742,0.446680,0.356187,0.353542,0.378493
43,0.692000,0.462193,0.558725,0.549306,0.139964,0.367555,0.620495,0.425468,0.524047,0.425468,...,0.507529,0.625275,0.485153,0.412742,0.446680,0.356187,0.353542,0.378493,0.707161,0.528649
44,0.558725,0.549306,0.139964,0.367555,0.620495,0.425468,0.524047,0.425468,0.567939,0.425468,...,0.485153,0.412742,0.446680,0.356187,0.353542,0.378493,0.707161,0.528649,0.477008,0.270177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,0.476575,0.659079,0.587572,0.341533,0.664741,0.288187,0.532863,0.315916,0.561435,0.409821,...,0.569342,0.409821,0.753237,0.409290,0.525512,0.373358,0.380016,0.425468,0.334770,0.493755
1468,0.587572,0.341533,0.664741,0.288187,0.532863,0.315916,0.561435,0.409821,0.576284,0.499028,...,0.753237,0.409290,0.525512,0.373358,0.380016,0.425468,0.334770,0.493755,0.429987,0.392545
1469,0.664741,0.288187,0.532863,0.315916,0.561435,0.409821,0.576284,0.499028,0.561040,0.438911,...,0.525512,0.373358,0.380016,0.425468,0.334770,0.493755,0.429987,0.392545,0.568702,0.442213
1470,0.532863,0.315916,0.561435,0.409821,0.576284,0.499028,0.561040,0.438911,0.906212,0.572588,...,0.380016,0.425468,0.334770,0.493755,0.429987,0.392545,0.568702,0.442213,0.274459,0.487127


In [48]:
X_train[0]

array([[0.59858304, 0.48397333, 0.55699928, 0.47757745, 0.61853723,
        0.35464231, 0.69199978, 0.46219345, 0.55872546, 0.54930582],
       [0.13996351, 0.36755485, 0.62049499, 0.42546788, 0.52404684,
        0.42546788, 0.56793922, 0.42546788, 0.62374806, 0.37689749]])

In [57]:
from pandas import DataFrame, concat
def series_to_supervised(data, n_in=10, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [58]:
series_to_supervised(dataset)

,var1(t-10),var2(t-10),var1(t-9),var2(t-9),var1(t-8),var2(t-8),var1(t-7),var2(t-7),var1(t-6),var2(t-6),...,var1(t-4),var2(t-4),var1(t-3),var2(t-3),var1(t-2),var2(t-2),var1(t-1),var2(t-1),var1(t),var2(t)
10,0.598583,0.483973,0.556999,0.477577,0.618537,0.354642,0.692000,0.462193,0.558725,0.549306,...,0.620495,0.425468,0.524047,0.425468,0.567939,0.425468,0.623748,0.376897,0.390033,0.408113
11,0.556999,0.477577,0.618537,0.354642,0.692000,0.462193,0.558725,0.549306,0.139964,0.367555,...,0.524047,0.425468,0.567939,0.425468,0.623748,0.376897,0.390033,0.408113,0.457607,0.425468
12,0.618537,0.354642,0.692000,0.462193,0.558725,0.549306,0.139964,0.367555,0.620495,0.425468,...,0.567939,0.425468,0.623748,0.376897,0.390033,0.408113,0.457607,0.425468,0.492984,0.442822
13,0.692000,0.462193,0.558725,0.549306,0.139964,0.367555,0.620495,0.425468,0.524047,0.425468,...,0.623748,0.376897,0.390033,0.408113,0.457607,0.425468,0.492984,0.442822,0.583957,0.425468
14,0.558725,0.549306,0.139964,0.367555,0.620495,0.425468,0.524047,0.425468,0.567939,0.425468,...,0.390033,0.408113,0.457607,0.425468,0.492984,0.442822,0.583957,0.425468,0.449065,0.503873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099,0.524357,0.448750,0.438987,0.489189,0.251956,0.577433,0.479703,0.505186,0.417643,0.387272,...,0.511901,0.344393,0.475984,0.388742,0.563457,0.496293,0.526293,0.408722,0.526307,0.371388
2100,0.438987,0.489189,0.251956,0.577433,0.479703,0.505186,0.417643,0.387272,0.548542,0.369262,...,0.475984,0.388742,0.563457,0.496293,0.526293,0.408722,0.526307,0.371388,0.478919,0.444184
2101,0.251956,0.577433,0.479703,0.505186,0.417643,0.387272,0.548542,0.369262,0.511901,0.344393,...,0.563457,0.496293,0.526293,0.408722,0.526307,0.371388,0.478919,0.444184,0.499266,0.366962
2102,0.479703,0.505186,0.417643,0.387272,0.548542,0.369262,0.511901,0.344393,0.475984,0.388742,...,0.526293,0.408722,0.526307,0.371388,0.478919,0.444184,0.499266,0.366962,0.499069,0.382056
